# Bibliotecas

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
import sqlite3
from datetime import datetime
import json
from sentence_transformers import SentenceTransformer, util
import torch

# Credenciais

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('api_key')
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
model = os.environ["OPENAI_MODEL_NAME"]

# SQLite

In [4]:
DB_FILE = "../db/historico.db"

def setup_database():
    """
    Cria o banco de dados e a tabela.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS interacoes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT NOT NULL, 
            prompt_usuario TEXT NOT NULL,
            resposta_agente TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

def log_interaction(prompt, response):
    """
    Salva a interação no banco de dados.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute("""
        INSERT INTO interacoes (timestamp, prompt_usuario, resposta_agente)
        VALUES (?, ?, ?)
    """, (timestamp_str, prompt, response))
    
    conn.commit()
    conn.close()
    print("\n[LOG] Interação registrada com sucesso no banco de dados.")

# Consulta de catálogo

In [10]:
# Carregamento do catálogo
with open("../data/dados-produtos.json", "r", encoding="utf-8") as f:
    produtos = json.load(f)

# Preparação para busca com embeddings
modelo = SentenceTransformer('all-MiniLM-L6-v2')
corpus = [f"{p['title']} {p['description']}" for p in produtos]
corpus_embeddings = modelo.encode(corpus, convert_to_tensor=True)

# ========================================
# Ferramenta personalizada com embeddings
class ProductSearchTool(BaseTool):
    name: str = "Busca de Produtos no Catálogo"
    description: str = "Busca produtos no catálogo com base em similaridade semântica com a consulta do cliente."

    def _run(self, query: str) -> str:
        consulta_embedding = modelo.encode(query, convert_to_tensor=True)
        similaridades = util.cos_sim(consulta_embedding, corpus_embeddings)[0]

        top_indices = torch.topk(similaridades, k=5).indices  # Retorna os 5 mais semelhantes
        resultados = [produtos[i] for i in top_indices if similaridades[i] > 0.4]

        if not resultados:
            return "❌ Nenhum produto encontrado com essa descrição."

        resposta = ""
        for p in resultados:
            resposta += (
                f"\n🛍️ *{p['title']}*\n"
                f"💲 R$ {p['price']:.2f}\n"
                f"📄 {p['description']}\n"
                f"🖼️ {p['image']}\n"
            )

        return resposta.strip()

# ========================================
# Instancia a ferramenta
catalogo_tool = ProductSearchTool()

# ========================================
# Agente de atendimento do catálogo
agente_catalogo = Agent(
    role="Especialista de Catálogo de Produtos",
    goal="Ajudar o cliente a encontrar produtos desejados com base em sua busca.",
    backstory=(
        "Você é um especialista em produtos da C&A, responsável por identificar os melhores itens com base em palavras-chave fornecidas pelo cliente."
    ),
    tools=[catalogo_tool],
    allow_delegation=False,
    llm=model
)

# ========================================
# Tarefa principal
tarefa_busca_produto = Task(
    description="Um cliente quer encontrar um produto com a descrição: '{busca}'. Use a ferramenta para encontrar os produtos ideais.",
    expected_output="Uma lista de produtos com nome, preço, descrição e imagem.",
    agent=agente_catalogo
)

# ========================================
# Crew
crew_catalogo = Crew(
    agents=[agente_catalogo],
    tasks=[tarefa_busca_produto],
    process=Process.sequential
)

# ========================================
# Execução

setup_database()

busca = "Calça azul moletom"
result = crew_catalogo.kickoff(inputs={"busca": busca})

log_interaction(prompt=busca, response=str(result))

print("\n========================")
print("Resultado da busca:")
print("========================\n")
print(result)


[LOG] Interação registrada com sucesso no banco de dados.

Resultado da busca:

1. Nome: Calça de Moletom Infantil Azul Marinho  
   Preço: R$ 139.99  
   Descrição: Calça moletom infantil azul marinho, roupas, calças tamanhos 4, 6, 8, 10, 12  
   Imagem: https://cea.vteximg.com.br/arquivos/ids/58596058/Foto-0.jpg?v=638500098037070000  

2. Nome: Calça Slim de Sarja Azul Marinho  
   Preço: R$ 179.99  
   Descrição: Calça slim sarja azul marinho, moda masculina, roupas, calças tamanhos 38, 40, 42, 44, 46, 48, 50, 52  
   Imagem: https://cea.vteximg.com.br/arquivos/ids/53567879/calca-slim-de-sarja-azul-marinho-1023267-Azul_Marinho_1.jpg?v=637828881471030000  

3. Nome: Calça Reta de Sarja Azul  
   Preço: R$ 239.99  
   Descrição: Calça reta sarja azul, moda masculina, roupas, calças tamanhos 38, 40, 42, 44, 46  
   Imagem: https://cea.vteximg.com.br/arquivos/ids/59168355/Foto-0.jpg?v=638687569029430000  

4. Nome: Calça de Moletom Infantil com Bolsos Azul Marinho  
   Preço: R$ 99.99 

# Consulta SQLite

In [9]:
conn = sqlite3.connect('../db/historico.db')
cursor = conn.cursor()

print("--- Histórico de Interações ---")
for row in cursor.execute("SELECT id, timestamp, prompt_usuario, resposta_agente FROM interacoes ORDER BY timestamp DESC"):
    print(f"\nID: {row[0]}")
    print(f"Data/Hora: {row[1]}")
    print(f"Usuário Perguntou: {row[2]}")
    print(f"Agente Respondeu: {row[3]}")
    print("-" * 20)

conn.close()

--- Histórico de Interações ---

ID: 4
Data/Hora: 2025-06-25 19:56:05
Usuário Perguntou: Calça preta masculina
Agente Respondeu: 1. Nome: Calça Super Skinny Preto
   Preço: R$ 159.99
   Descrição: Calça super skinny preta moda masculina, roupas calças tamanhos 36, 38, 40, 42, 44, 46, 48
   Imagem: ![Imagem](https://cea.vteximg.com.br/arquivos/ids/53413308/CALCA-SUPER-SKINNY-PADUA-BLACK-AMACIADA-LISA-1023252-1023252_1.jpg?v=637813254587170000)

2. Nome: Calça Reta de Moletom Preto
   Preço: R$ 119.99
   Descrição: Calça reta moletom preto moda masculina, roupas calças tamanhos P, M, G, GG
   Imagem: ![Imagem](https://cea.vteximg.com.br/arquivos/ids/58454728/Foto-1.jpg?v=638441237606300000)

3. Nome: Calça Slim de Sarja Preta
   Preço: R$ 139.99
   Descrição: Calça slim sarja preta moda masculina, roupas calças tamanhos 36, 38, 40, 42, 44, 46, 48
   Imagem: ![Imagem](https://cea.vteximg.com.br/arquivos/ids/55152963/calca-slim-de-sarja-preta-1029819-Preto_1.jpg?v=637934307487770000)

4. N